In [19]:
import os
from os import path
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), '../..'))

import db_builder.db_handler as dbh

import numpy as np
import cv2
from torchvision import transforms
from tqdm import tqdm
import datetime

In [20]:
db_params = {
    'user': 'postgres',
    'password': input('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase', #tensionTerminator
}

Please enter DB pw DiBSE21


In [21]:
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
filepath_with_middle = toolcheck.get_filepath_with_middle()
filepath_with_rightside = toolcheck.get_filepath_with_rightside()
filepath_with_leftside = toolcheck.get_filepath_with_leftside()
toolcheck.disconnect()

Connected to PostgreSQL, DB: ttdatabase
2023-11-20 20:33:30,443 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-20 20:33:30,444 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-20 20:33:30,445 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-20 20:33:30,445 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-20 20:33:30,446 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-20 20:33:30,446 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-20 20:33:30,447 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-20 20:33:30,449 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop JOIN bodyside_loop_association ON loop.id = bodyside_loop_association.loop_id 
WHERE loop.human_labeled = true AND bodyside_loop_association.bodyside_id = %(bodyside_id

In [22]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [23]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 1

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [24]:
def get_time():
    return datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

In [25]:
def save_path(label: str, time):
    abs_path = os.getcwd()
    two_up =  path.abspath(path.join(abs_path ,"../.."))
    return path.join(two_up, 'data', f'{os.path.basename(abs_path)}', f'{time}', label)

In [26]:
current_time = get_time()

In [27]:
for x in tqdm(filepath_with_rightside):
    video_to_image_converter(x, save_path('right', current_time))

100%|██████████| 37/37 [1:09:29<00:00, 112.69s/it]


In [28]:
for x in tqdm(filepath_with_middle):
    video_to_image_converter(x, save_path('middle', current_time))

100%|██████████| 56/56 [1:08:50<00:00, 73.77s/it]


In [29]:
for x in tqdm(filepath_with_leftside):
    video_to_image_converter(x, save_path('left', current_time))

100%|██████████| 35/35 [1:14:28<00:00, 127.68s/it]
